In [94]:
import torch
from torch import nn, optim
import torch.nn.functional as F
# from torchtext.legacy import data
import numpy as np
import pandas as pd
import math
import time
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [95]:
# init
seed = 1
batch_size = 128
learning_rate = 1e-3

# steup using GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# setuo random seed
torch.manual_seed(seed)
time_step = 7


In [175]:
# load FinTech data
df = pd.read_csv('../data/FinTech.csv', encoding='utf-8')
df.head()

,Sentiment_24h,DXYUSD_Gain_Rate,World_Index_Gain_Rate,GoldUSD_Gain_Rate,Silver_Gain_Rate,DBCCommodity_Gain_Rate,DJCI_Gain_Rate,CrudeOil_Gain_Rate,SPY_Gain_Rate,VIX_Gain_Rate,TNXTbill_Gain_Rate,BTC_Price
0,0.188838,0.001341,0.003931,0.006341,0.006341,0.008765,0.019090,0.019418,0.003971,-0.000729,0.003007,117.00
1,0.169673,-0.005234,0.007311,-0.014732,-0.014732,-0.007181,-0.001202,-0.010167,0.002574,-0.013858,0.014537,103.43
2,0.131704,-0.000857,0.000000,-0.038969,-0.038969,-0.000387,-0.002442,-0.022024,-0.006590,0.043948,-0.011460,91.01
3,0.128927,0.007229,0.004812,0.009762,0.009762,0.008491,0.029350,0.032857,0.006743,-0.061464,-0.007304,118.33
4,0.157061,-0.001460,-0.012230,0.009474,0.009474,0.001896,-0.000804,0.017236,0.001427,-0.005418,0.076829,106.40


In [106]:
# Divide the data set into inputs and outputs
df_input = df.drop(['BTC_Price'], axis=1).values
df_ouput = df[['BTC_Price']].values

n_samples, n_features = df_input.shape
df_input.shape, df_ouput.shape

((1912, 11), (1912, 1))

In [107]:
# Normalized data
X_scaler = StandardScaler()
Y_scaler = StandardScaler()
X_scaler = X_scaler.fit(df_input)
df_input = X_scaler.transform(df_input)
# X_test = X_scaler.transform(X_test)

Y_scaler = Y_scaler.fit(df_ouput)
df_ouput = Y_scaler.transform(df_ouput)
# Y_test = Y_scaler.transform(Y_test)
# X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [108]:
# built timeseries data for LSTM
X_data = np.zeros([n_samples-time_step+1, time_step, n_features])
Y_data = torch.Tensor(df_ouput[time_step-1:, :])
n_samples = X_data.shape[0]
X_data.shape, Y_data.shape

for i in range(n_samples):
    X_data[i, :, :] = df_input[i:i+time_step, :]
    
X_data = torch.Tensor(X_data)
X_data.shape, Y_data.shape

(torch.Size([1906, 7, 11]), torch.Size([1906, 1]))

In [109]:
# split data to train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.25, random_state=1)
# make DataLoader
train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, Y_test), batch_size, shuffle=False)
# for bath_id, batch_data in enumerate(train_loader):
#     print(bath_id, batch_data[0].shape)   # trian_data
#     print(bath_id, batch_data[1].shape)   # test_data

In [129]:
class LongShortTermMemory(nn.Module):
    def __init__(self, timestep, n_features, hidden_size, n_layers):
        super(LongShortTermMemory, self).__init__()
        
        self.hidden_size = hidden_size
        self.timestep = timestep
        self.n_layers = n_layers
        
        # built LSTM model
        self.lstm = nn.LSTM(input_size=timestep, 
                            hidden_size=hidden_size,
                            num_layers=n_layers,
                            bias=True,
                            batch_first=False,
                            dropout=0.5,
                            bidirectional=False
                           )
        self.num_directions = 2 if self.lstm.bidirectional else 1
        self.fc = nn.Linear(hidden_size*self.num_directions, 1)
        self.dropout = nn.Dropout(0.2)
        # attention net parameter
        self.atten_w = nn.Parameter(torch.Tensor(hidden_size*self.num_directions, hidden_size*self.num_directions))
        self.atten_u = nn.Parameter(torch.Tensor(hidden_size*self.num_directions, 1))
        
        # init parameters
        nn.init.uniform_(self.atten_w, -0.1, 0.1)
        nn.init.uniform_(self.atten_u, -0.1, 0.1)
        
    def attention_net(self, lstm_output, hidden_state):
#         u: [batch_size, num_features, 2 * num_hiddens]
        u = torch.relu(torch.matmul(lstm_output, self.atten_w))
        
        attention = torch.matmul(u, self.atten_u)
       # attention: [batch_size, attention, 1]
        attention_weight = F.softmax(attention, dim=1)
       # att_score: [batch_size, n_features, 1]
        context = lstm_output * attention_weight
        # context: [bath_size, hidden_size*self.num_directions]
        context = torch.sum(context, dim=1)

        return context, attention_weight
        
    
    def forward(self, x):
        x = x.permute(2,0,1)                  #[batch_size, timestep, n_features] -> [n_features, batch_size, timestep]
        
        # output: [n_features, batch_size, hidden_size * num_directions], hidden_state: [num_directions * num_layers, batch, hidden_size]
        output, (hidden_state, cell_state) = self.lstm(x)
#         print("LSTM_out:", output.shape, hidden_state.shape, cell_state.shape)
        output = output.permute(1, 0, 2)                  #[batch_size, n_features, hidden_size]
#         query = self.dropout(output)
        attn_output, attention_weight = self.attention_net(output, hidden_state)
    
#         print("fc_input:", attn_output.shape)
        logit = self.fc(attn_output)
#         print("logit:", logit.shape)
        logit = self.dropout(logit)
    
        return logit, attention_weight

In [207]:
# built LSTM model
rnn = LongShortTermMemory(time_step, n_features, hidden_size=300, n_layers=2).to(device)

optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)
criteon = nn.MSELoss()

# 'R2 score'
def binary_acc(preds, y):
    u = torch.sum(torch.square(y - preds))
    v = torch.sum(torch.square(y - torch.mean(y)))
    return 1 - (u/v)


def train(rnn, iterator, optimizer, criteon):

    avg_loss = []
    avg_acc = []
    rnn.train() 

    for batch_id, batch_data in enumerate(iterator):
        inputs, labels = batch_data
        inputs = inputs.to(device)
        labels = labels.to(device)
        pred, _ = rnn(inputs)
#         print(pred.shape, labels.shape)
        loss = criteon(pred.squeeze(), labels.squeeze())
        acc = binary_acc(pred.squeeze(), labels.squeeze()).item()

        avg_loss.append(loss.item())
        avg_acc.append(acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_acc = np.array(avg_acc).mean()
    avg_loss = np.array(avg_loss).mean()
    return avg_loss, avg_acc


def evaluate(rnn, iterator, criteon):

    avg_loss = []
    avg_acc = []
    rnn.eval()
    att_weights = []
    with torch.no_grad():
        for batch_data in iterator:
            inputs, labels = batch_data
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            pred, attention_weight = rnn(inputs)
            att_weights.append(attention_weight.squeeze())
            loss = criteon(pred.squeeze(), labels.squeeze())
            acc = binary_acc(pred, labels).item()

            avg_loss.append(loss.item())
            avg_acc.append(acc)

    avg_loss = np.array(avg_loss).mean()
    avg_acc = np.array(avg_acc).mean()
    return avg_loss, avg_acc, att_weights


#train and print model
best_valid_acc = float('-inf')

epochs = 300
for epoch in range(epochs):

    start_time = time.time()

    train_loss, train_acc = train(rnn, train_loader, optimizer, criteon)
    dev_loss, dev_acc, att_weights = evaluate(rnn, test_loader, criteon)

    end_time = time.time()

    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)

    if dev_acc > best_valid_acc:          #只要模型效果变好，就保存
        best_valid_acc = dev_acc
        torch.save(rnn.state_dict(), 'lstm-model.pt')
    if epoch % 10 == 0 or epoch == epochs-1:
        print(f'Epoch: {epoch+1:2} | Epoch Time: {epoch_mins}m {epoch_secs:.2f}s')
        print(f'\tTrain Loss: {train_loss:.6f} | Train Acc: {train_acc*100:.5f}%')
        print(f'\t Val. Loss: {dev_loss:.6f} |  Val. Acc: {dev_acc*100:.5f}%')


# #use srored model to predice 
# rnn.load_state_dict(torch.load("lstm-model.pt"))
# test_loss, test_acc, att_weights = evaluate(rnn, test_iterator, criteon)
# print(f'Test. Loss: {test_loss:.3f} |  Test. Acc: {test_acc*100:.2f}%')

Epoch:  1 | Epoch Time: 0.0m 0.14s
	Train Loss: 0.806613 | Train Acc: 13.26372%
	 Val. Loss: 0.443042 |  Val. Acc: 58.37578%
Epoch: 11 | Epoch Time: 0.0m 0.12s
	Train Loss: 0.371108 | Train Acc: 61.10159%
	 Val. Loss: 0.255465 |  Val. Acc: 75.97686%
Epoch: 21 | Epoch Time: 0.0m 0.12s
	Train Loss: 0.340900 | Train Acc: 63.35112%
	 Val. Loss: 0.220810 |  Val. Acc: 79.21442%
Epoch: 31 | Epoch Time: 0.0m 0.12s
	Train Loss: 0.296224 | Train Acc: 69.29164%
	 Val. Loss: 0.205755 |  Val. Acc: 80.51293%
Epoch: 41 | Epoch Time: 0.0m 0.11s
	Train Loss: 0.278187 | Train Acc: 71.22802%
	 Val. Loss: 0.198906 |  Val. Acc: 81.09850%
Epoch: 51 | Epoch Time: 0.0m 0.10s
	Train Loss: 0.275303 | Train Acc: 71.07424%
	 Val. Loss: 0.187694 |  Val. Acc: 82.23489%
Epoch: 61 | Epoch Time: 0.0m 0.10s
	Train Loss: 0.300140 | Train Acc: 68.68736%
	 Val. Loss: 0.185539 |  Val. Acc: 82.51055%
Epoch: 71 | Epoch Time: 0.0m 0.11s
	Train Loss: 0.269542 | Train Acc: 72.13718%
	 Val. Loss: 0.193190 |  Val. Acc: 81.84783%


In [157]:
dev_loss, dev_acc, att_weights = evaluate(rnn, test_loader, criteon)

In [206]:
att = torch.cat(att_weights, dim=0).cpu()
pd.DataFrame(att.numpy().sum(axis=0).reshape(-1, 1))
df_coef = pd.DataFrame(columns=['variable', 'coefficient'])
df_coef['variable'] = df.columns[:-1]
df_coef['coefficient'] = att.numpy().sum(axis=0)
# df_coef['abs_coefficient'] = df_coef['coefficient'].abs()
# sort by absolute value of coefficient
pd.set_option('display.max_rows',None)
df_coef.sort_values(by=['coefficient'], ascending=False)

,variable,coefficient
0,Sentiment_24h,333.603882
1,DXYUSD_Gain_Rate,85.545662
10,TNXTbill_Gain_Rate,10.288287
9,VIX_Gain_Rate,9.721036
2,World_Index_Gain_Rate,9.524199
8,SPY_Gain_Rate,8.597301
7,CrudeOil_Gain_Rate,6.869552
6,DJCI_Gain_Rate,5.096609
5,DBCCommodity_Gain_Rate,3.469755
4,Silver_Gain_Rate,2.230522
